### *Imports*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

### *Get Data*

In [2]:
df = pd.read_excel("../input/productos-consumo-masivo/output - Kaggle.xlsx")

In [3]:
df.shape

(25638, 13)

In [4]:
df.columns

Index(['date', 'prod_id', 'prod_name', 'prod_name_long', 'prod_brand',
       'category', 'subcategory', 'tags', 'prod_unit_price', 'prod_units',
       'prod_icon', 'prod_source', 'source_type'],
      dtype='object')

In [5]:
df.head()

,date,prod_id,prod_name,prod_name_long,prod_brand,category,subcategory,tags,prod_unit_price,prod_units,prod_icon,prod_source,source_type
0,20190609,7702155021451,Limpia tapicería en espuma Binner,Limpia tapicería en espuma Binner x 400ml,BINNER,Supermercado,Aseo de hogar,Alfombras,15590,Unidades,../icons/SVG/53-places/home-3.svg,VERDE,1
1,20190609,7702155016457,Limpia tapicería y alfombras Binner,Limpia tapicería y alfombras Binner x 500ml,BINNER,Supermercado,Aseo de hogar,Alfombras,13993,Unidades,../icons/SVG/53-places/home-3.svg,VERDE,1
2,20190609,4008455446318,Limpiador Dr. Beckmann alfombras 2en1 poder oxi,Limpiador Dr. Beckmann alfombras 2en1 poder ox...,DR.BECKMANN,Supermercado,Aseo de hogar,Alfombras,16513,Unidades,../icons/SVG/53-places/home-3.svg,VERDE,1
3,20190609,7702532312677,Aceite Bon Aire eléctrico perfume flores,Aceite Bon Aire eléctrico perfume flores x 25 ml,BONAIRE,Supermercado,Aseo de hogar,Ambientadores,14990,Unidades,../icons/SVG/58-beauty-spas/spa-vapor.svg,VERDE,1
4,20190609,7702532312967,Ambientador Aire Varitas Bambú Bonaire 2 Repue...,Ambientador Aire Varitas Bambú Bonaire 2 Repue...,BONAIRE,Supermercado,Aseo de hogar,Ambientadores,34990,Unidades,../icons/SVG/58-beauty-spas/spa-vapor.svg,VERDE,1


In [6]:
df.dtypes

date                int64
prod_id             int64
prod_name          object
prod_name_long     object
prod_brand         object
category           object
subcategory        object
tags               object
prod_unit_price    object
prod_units         object
prod_icon          object
prod_source        object
source_type         int64
dtype: object

In [7]:
df.describe()

,date,prod_id,source_type
count,2.563800e+04,2.563800e+04,25638.0
mean,2.019279e+07,7.137052e+12,1.0
std,3.867261e+03,1.872051e+12,0.0
min,2.019061e+07,5.244812e+06,1.0
25%,2.019061e+07,7.702010e+12,1.0
50%,2.019071e+07,7.702177e+12,1.0
75%,2.019110e+07,7.703616e+12,1.0
max,2.020022e+07,9.333527e+12,1.0


# *1. Data Preparation*

Let's eliminate the column of icons, since it is not useful for this analysis.

In [8]:
df = df[['date', 
         'prod_id', 
         'prod_name', 
         'prod_name_long', 
         'prod_brand',
         'category', 
         'subcategory', 
         'tags', 
         'prod_unit_price', 
         'prod_units',
         # 'prod_icon', 
         'prod_source', 
         'source_type'
        ]].copy()
df.shape

(25638, 12)

The date column is as an integer, so let's change it to be date type

In [9]:
df['date'] = pd.to_datetime(df['date'].astype(str), format='%Y%m%d')

In [10]:
df.dtypes

date               datetime64[ns]
prod_id                     int64
prod_name                  object
prod_name_long             object
prod_brand                 object
category                   object
subcategory                object
tags                       object
prod_unit_price            object
prod_units                 object
prod_source                object
source_type                 int64
dtype: object

Let's see if there is data loss in our data set

In [11]:
df.isna().sum()

date                 0
prod_id              0
prod_name          132
prod_name_long       0
prod_brand          52
category             0
subcategory          0
tags                 0
prod_unit_price      5
prod_units         407
prod_source          0
source_type          0
dtype: int64

Since we have some N/A product names, we will eliminate this column to work with the long product name column that has no N/A data.

In [12]:
df = df.drop(['prod_name'], axis = 1).copy()

In [13]:
df['category'].unique()

array(['Supermercado'], dtype=object)

Let's remove the category column, as there is only one category

In [14]:
df = df.drop(['category'], axis = 1).copy()

Now let's review the product units in N/A and why this is happening.

In [15]:
df['prod_units'].unique()

array(['Unidades', 'Botella', nan], dtype=object)

In [16]:
df.loc[pd.isna(df['prod_units'])].head()

,date,prod_id,prod_name_long,prod_brand,subcategory,tags,prod_unit_price,prod_units,prod_source,source_type
20205,2020-02-20,7501058715289,Aromatizante Air Wick Pure Repuesto Suavidad x...,AIR WICK,Aseo de hogar,Ambientadores,19590,NaN,VERDE,1
20360,2020-02-20,7591005992894,Insecticida 45 noches repuesto raid x 2 un x 6...,RAID,Aseo de hogar,Insecticidas,25990,NaN,VERDE,1
20396,2020-02-20,7591005981195,Limpia Pisos Campos De Lavanda Mr Musculo x 18...,MR MUSCULO,Aseo de hogar,Desinfectantes,8390,NaN,VERDE,1
20443,2020-02-20,7702155048779,Limpiador de piso brillo desinfectante lavanda...,BINNER,Aseo de hogar,Desinfectantes,12990,NaN,VERDE,1
20507,2020-02-20,7702535013830,Agua Brisa botella 1 litro - Agua Brisa en bot...,BRISA,Bebidas,Agua,2150,NaN,VERDE,1


In [17]:
df.loc[df['prod_id'] == 7501058715289].head()

,date,prod_id,prod_name_long,prod_brand,subcategory,tags,prod_unit_price,prod_units,prod_source,source_type
43,2019-06-09,7501058715289,Aromatizante Air Wick Pure repuesto suavidad x...,AIR WICK,Aseo de hogar,Ambientadores,13713,Unidades,VERDE,1
8011,2019-07-09,7501058715289,Aromatizante Air Wick Pure repuesto suavidad x...,AIR WICK,Aseo de hogar,Ambientadores,19590,Unidades,VERDE,1
14694,2019-11-01,7501058715289,Aromatizante Air Wick Pure repuesto suavidad x...,AIR WICK,Aseo de hogar,Ambientadores,19590,Unidades,VERDE,1
20205,2020-02-20,7501058715289,Aromatizante Air Wick Pure Repuesto Suavidad x...,AIR WICK,Aseo de hogar,Ambientadores,19590,NaN,VERDE,1


We can see that there are products that have their units defined, but not all of them have this value, so let's fix it.

In [18]:
df['prod_units'] = df.groupby('prod_id')['prod_units'].transform(lambda x: x.ffill().bfill()).astype(df['prod_units'].dtype)

let's apply this same solution for the product brand and unit price

In [19]:
df['prod_brand'] = df['prod_brand'].astype("string")
df['prod_brand'] = df.groupby('prod_id')['prod_brand'].transform(lambda x: x.ffill().bfill())
df['prod_unit_price'] = df['prod_unit_price'].astype("string")
df['prod_unit_price'] = df.groupby('prod_id')['prod_unit_price'].transform(lambda x: x.ffill().bfill())

In [20]:
df.isna().sum()

date                0
prod_id             0
prod_name_long      0
prod_brand         25
subcategory         0
tags                0
prod_unit_price     0
prod_units          0
prod_source         0
source_type         0
dtype: int64

We see that we reduce the amount of nan data in our data set quite a bit.

Let's see if there is duplicate data in our dataset

In [21]:
df.loc[df.duplicated(subset=['prod_id'])].head()

,date,prod_id,prod_name_long,prod_brand,subcategory,tags,prod_unit_price,prod_units,prod_source,source_type
2224,2019-06-09,7702425803992,Pañuelos Kleenex kids caja x12paquetes,KLEENEX,Cuidado Personal,Pañuelos,2583,Unidades,VERDE,1
2225,2019-06-09,7702425524828,Pañuelos Kleenex Original x 2 cubos x 60 und c-u,KLEENEX,Cuidado Personal,Pañuelos,5733,Unidades,VERDE,1
2226,2019-06-09,7702425805798,Pañuelos Kleenex triple hoja x 80unds precio e...,KLEENEX,Cuidado Personal,Pañuelos,3073,Unidades,VERDE,1
2238,2019-06-09,7702425803039,Papel higiénico SCOTT cuidado completo triple...,SCOTT,Cuidado Personal,Papel higiénico,4893,Unidades,VERDE,1
2715,2019-06-09,7702175130799,Pimentero Rey Ed Especial,EL REY,Despensa,"Condimentos, caldos y sal",6590,Unidades,VERDE,1


Let's take a closer look at why they are duplicated and whether they have a logical reason to be duplicated.

In [22]:
df.query('prod_id == 7702425803992')

,date,prod_id,prod_name_long,prod_brand,subcategory,tags,prod_unit_price,prod_units,prod_source,source_type
2221,2019-06-09,7702425803992,Pañuelos Kleenex kids caja x12paquetes,KLEENEX,Cuidado Personal,Pañuelos,2583,Unidades,VERDE,1
2224,2019-06-09,7702425803992,Pañuelos Kleenex kids caja x12paquetes,KLEENEX,Cuidado Personal,Pañuelos,2583,Unidades,VERDE,1
10225,2019-07-09,7702425803992,Pañuelos Kleenex kids caja x12paquetes,KLEENEX,Cuidado Personal,Pañuelos,2583,Unidades,VERDE,1
10228,2019-07-09,7702425803992,Pañuelos Kleenex kids caja x12paquetes,KLEENEX,Cuidado Personal,Pañuelos,3690,Unidades,VERDE,1
16497,2019-11-01,7702425803992,Pañuelos Kleenex kids caja x12paquetes,KLEENEX,Cuidado Personal,Pañuelos,3790,Unidades,VERDE,1
16500,2019-11-01,7702425803992,Pañuelos Kleenex kids caja x12paquetes,KLEENEX,Cuidado Personal,Pañuelos,3790,Unidades,VERDE,1
21885,2020-02-20,7702425803992,Pañuelos Kleenex kids caja x12paquetes,KLEENEX,Cuidado Personal,Pañuelos,3790,Unidades,VERDE,1


In [23]:
df.query('prod_id == 7702425803992 and date == @pd.Timestamp("2019-06-09")')

,date,prod_id,prod_name_long,prod_brand,subcategory,tags,prod_unit_price,prod_units,prod_source,source_type
2221,2019-06-09,7702425803992,Pañuelos Kleenex kids caja x12paquetes,KLEENEX,Cuidado Personal,Pañuelos,2583,Unidades,VERDE,1
2224,2019-06-09,7702425803992,Pañuelos Kleenex kids caja x12paquetes,KLEENEX,Cuidado Personal,Pañuelos,2583,Unidades,VERDE,1


In [24]:
df.query('prod_id == 7702425803992 and date == @pd.Timestamp("2019-11-01")')

,date,prod_id,prod_name_long,prod_brand,subcategory,tags,prod_unit_price,prod_units,prod_source,source_type
16497,2019-11-01,7702425803992,Pañuelos Kleenex kids caja x12paquetes,KLEENEX,Cuidado Personal,Pañuelos,3790,Unidades,VERDE,1
16500,2019-11-01,7702425803992,Pañuelos Kleenex kids caja x12paquetes,KLEENEX,Cuidado Personal,Pañuelos,3790,Unidades,VERDE,1


Being a data set of “available products” we can assume that the dates and ids of repeated products indicate the stock available at that date, so this should not cause any problems

# 2. Data Understanfing